In [106]:
import gurobipy as gp
from gurobipy import GRB

***Please refreing to this list for corresponding funitures***

'a1' = beds, 

'a2' = Bookcases, 

'a3' = chair_dining, 

'a4' = chair_living, 

'a5' = chair_office, 

'a6' = coffe_table, 

'a7' = couches, 

'a8' = Desk, 

'a9' = dining_table, 

'a10' = Dresser

In [107]:
# setting the profit for each furniture
furnitures, profit = gp.multidict({
    'a1' : 10, 
    'a2' : 7, 
    'a3' : 12, 
    'a4' : 9, 
    'a5' : 3, 
    'a6' : 5, 
    'a7' : 8, 
    'a8' : 6, 
    'a9' : 13, 
    'a10': 4
})

# list of furniture in each catefories
Categories = {
    'Bedrooms' : ['a1', 'a10'],
    'living_room' : ['a4', 'a6', 'a7'],
    'Dining' : ['a3', 'a9'],
    'office' : ['a2', 'a5', 'a8'],
    'beds' : ['a1'],
    'seating' : ['a3', 'a4', 'a5', 'a7'],
    'tables' : ['a6', 'a9'],
    'storage' : ['a2','a8', 'a10']
}

# minimum space for each categories
Categorie, percent_space = gp.multidict({
    'Bedrooms' : 0.2, 
    'living_room' : 0.2,
    'Dining': 0.15, 
    'office' : 0.10, 
    'beds': 0.10, 
    'seating': 0.50, 
    'tables': 0.10, 
    'storage' : 0.05
})

In [108]:
print(str(furnitures).lstrip(" "))

['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10']


In [109]:
model_profit  = gp.Model('furniture_profit')
# each features is percent space for each furniture 
space = model_profit.addVars(furnitures, name = 'space')

# maximiza profit
model_profit.setObjective(space.prod(profit), GRB.MAXIMIZE)

# each furniture have at least 0.05 space
model_profit.addConstrs((space[i] >= 0.05 for i in furnitures), name = 'minimum_space')
# space of a9 = 1.5 a3
model_profit.addConstr(space['a9'] == 1.5 * space['a3'], name = 'dining_space')

# loop through the condition for each categories
for i in Categories.keys():
    # adding the sapce considered in each categories
    model_profit.addConstr(gp.quicksum(space[j] for j in Categories[i]) >= percent_space[i], name =f'min_{i}')

# the total of percent space is smaller than 1 
model_profit.addConstr(gp.quicksum(space[i] for i in furnitures) ==1, name = 'c3')



<gurobi.Constr *Awaiting Model Update*>

In [110]:
model_profit.optimize()
if model_profit.Status == GRB.OPTIMAL:
    for i in furnitures:
        print(f"""{i}: {space[i].X}  """)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 20 rows, 10 columns and 42 nonzeros
Model fingerprint: 0x78980f0c
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 1e+00]
Presolve removed 17 rows and 6 columns
Presolve time: 0.02s
Presolved: 3 rows, 4 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0925600e+01   3.014000e-01   0.000000e+00      0s
       2    9.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  9.000000000e+00
a1: 0.15000000000000002  
a2: 0.05  
a3: 0.0999999999999998  
a4: 0.3000000000000002  
a5: 0.05  
a6: 0.05  
a7: 0.05  
a8: 0.05  
a9: 0.14999999999